# Unlocking the Palate - Evaluating Taste Consensus Among Beer Reviewers

---

Group [**BlackAda**](https://en.wikipedia.org/wiki/Blackadder)

> - Ludek Cizinsky ([ludek@cizinsky@epfl.ch](ludek.cizinsky@epfl.ch))
> - Peter Nutter ([peter@nutter@epfl.ch](peter@nutter@epfl.ch))
> - Pierre Lardet ([pierre@lardet@epfl.ch](pierre@lardet@epfl.ch))
> - Christian Bastin ([christian@bastin@epfl.ch](christian@bastin@epfl.ch))
> - Mika Senghaas ([mika@senghaas@epfl.ch](mika@senghaas@epfl.ch))

## Introduction

---

Navigating the world of beer reviews can be a daunting task for non-experts. Beer aficionados often describe brews as having nuanced flavors such as "grassy notes" and "biscuity/ crackery malt," with hints of "hay." But do these descriptions reflect the actual tasting experience? Following a "wisdom-of-the-crowd" approach, a descriptor can be considered meaningful if many, independent reviewers use similar descriptors for a beer's taste. To quantify consensus, we use natural language processing techniques to extract descriptors of a beer's taste and numerically represent these descriptors to compute similarity or consensus scores. The consensus scores between beer reviews will unveil whether there is a shared understanding of taste among beer geeks.

## Dependencies

---

We load the dependencies required for this project to run.

In [2]:
# Enable continuous module reloading
%load_ext autoreload
%autoreload 2 
%aimport src

# Standard library
import os

# Custom modules
from src import utils

And set some global variables.

In [3]:
# URL for the full dataset
DATA_URL = "https://drive.google.com/u/2/uc?id=1IqcAJtYrDB1j40rBY5M-PGp6KNX-E3xq&export=download"

# Number of samples to use for the subset
NUM_SUBSET_SAMPLES = 100

# Paths
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data")

# Ensure data directory exists
os.makedirs(DATA_DIR, exist_ok=True)

## Data

---

We will be working with the beer review data from the [BeerAdvocate](https://www.beeradvocate.com/) platform. 


### Data Download

Due to its size (uncompressed 1.6 GB), the dataset is not included in the repository but must be downloaded. The course staff has provided the data via Google Drive. On the first run of this notebook, we download the compressed data file from Google Drive and extract it to the `data` folder. The compressed file is ~1.5 GB in size. 

After extraction and removing of unnecessary files (archives, ratings file, ...), the data folder should contain the following files: `beers.csv`, `breweries.csv`, `users.csv`, `reviews.txt`. The total size of the data is ~2.9 GB.

*NB: Data loading takes around **~8min** on the first run. Subsequent runs of this cell are instant.*

In [31]:
# Download the BeerAdvocate dataset if it doesn't exist
if not os.path.exists(os.path.join(DATA_DIR, "reviews.txt")):
    utils.download_data(DATA_URL, data_dir=DATA_DIR)
print(f"Beer reviews downloaded to {DATA_DIR} ✅.")

Beer reviews downloaded to /Users/peter/Developer/ada-2023-project-blackada/data ✅.


### Data Loading

Next, we load the data into a Pandas DataFrame. On the first run, we load all the reviews from the `reviews.txt` file and populate it with some additional meta-data from the other files. We then save the DataFrame to a `.feather` file for faster loading in the future. On subsequent runs, we load the DataFrame from the `.feather` file if it exists.

*NB: Running this cell for the first time reads in all `2.5M` reviews which takes **~7min**. Subsequent runs should be much faster, taking about **~1min**.*

In [4]:
# Load all reviews and a subset of reviews (10,000)
reviews = utils.load_data(DATA_DIR)
sub_reviews = utils.load_data(DATA_DIR, num_samples=NUM_SUBSET_SAMPLES)

print(f"Loaded {len(reviews)} reviews ✅. (+{len(sub_reviews)} reviews in subset)")

Loaded 2589586 reviews ✅. (+100 reviews in subset)


### EDA

Let's explore the data a bit. We will analyse:

- ...

In [33]:
# TODO

In [10]:
from src.consensus import CosineSimilarity
from src.extractors import *
from src.pipeline import TextAnalysis
from src.embedders import *
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
lemma_extractor = LemmaExtractor()
adjective_extractor = AdjectiveExtractor()
dummy_extractor = DummyExtractor()

count_embedder = CountEmbedder()
tfidf_embedder = TfidfEmbedder()
bert_embedder = BertEmbedder()
sentence_transformer = SentenceTransformerEmbedder()
cosine_metric = CosineSimilarity()

lemma_and_count = TextAnalysis(lemma_extractor, count_embedder, cosine_metric)
lemma_and_tfidf = TextAnalysis(lemma_extractor, tfidf_embedder, cosine_metric)
adj_and_count = TextAnalysis(adjective_extractor, count_embedder, cosine_metric)
adj_and_tfidf = TextAnalysis(adjective_extractor, tfidf_embedder, cosine_metric)
dummy_and_bert = TextAnalysis(dummy_extractor, bert_embedder, cosine_metric)
dummy_and_sentence = TextAnalysis(dummy_extractor, sentence_transformer, cosine_metric)


def run_pipelines(docs, pipelines):
    results = {}
    for name, pipeline in pipelines.items():
        results[name] = pipeline.transform(docs)
    return results


pipelines = {
    "Lemma & Count": lemma_and_count,
    "Lemma & TFIDF": lemma_and_tfidf,
    "Adjective & Count": adj_and_count,
    "Adjective & TFIDF": adj_and_tfidf,
    "Dummy & BERT": dummy_and_bert,
    "Dummy & Senence Transformer": dummy_and_sentence,
}

In [6]:
most_popular_styles = reviews["beer"]["style"].value_counts()
most_popular_beers = reviews["beer"]["name"].value_counts()

In [12]:
style_results = []
for style in most_popular_styles.index[:10]:
    sample = reviews[reviews["beer"]["style"] == style].sample(100, random_state=SEED)
    transform = sample["review"]["text"].apply(lambda x: nlp(x))
    result = run_pipelines(transform, pipelines)
    style_results.append(result)
df_style_results = pd.DataFrame(style_results)
df_style_results

,Lemma & Count,Lemma & TFIDF,Adjective & Count,Adjective & TFIDF,Dummy & BERT,Dummy & Senence Transformer
0,0.387268,0.151790,0.119081,0.067537,0.958266,0.596320
1,0.439708,0.169459,0.127178,0.068523,0.958078,0.577297
2,0.418018,0.161084,0.157012,0.079044,0.957206,0.564650
3,0.402188,0.155264,0.127014,0.074352,0.953592,0.564476
4,0.432146,0.170018,0.166042,0.084089,0.953939,0.591735
5,0.424493,0.170841,0.151555,0.079022,0.942794,0.547803
6,0.420441,0.155198,0.132478,0.074572,0.950007,0.562347
7,0.406826,0.160338,0.132015,0.072228,0.955128,0.587860
8,0.422228,0.149259,0.094405,0.054850,0.951311,0.527485
9,0.402771,0.160068,0.163924,0.085475,0.946440,0.550708


In [13]:
beer_results = []
for beer in most_popular_beers.index[:10]:
    sample = reviews[reviews["beer"]["name"] == beer].sample(100, random_state=SEED)
    transform = sample["review"]["text"].apply(lambda x: nlp(x))
    result = run_pipelines(transform, pipelines)
    beer_results.append(result)
df_beer_results = pd.DataFrame(beer_results)
df_beer_results

,Lemma & Count,Lemma & TFIDF,Adjective & Count,Adjective & TFIDF,Dummy & BERT,Dummy & Senence Transformer
0,0.462031,0.190844,0.139913,0.078778,0.956497,0.597419
1,0.463539,0.195457,0.165077,0.087145,0.954542,0.608977
2,0.444459,0.171863,0.171805,0.085576,0.958744,0.585028
3,0.478463,0.198903,0.127421,0.074717,0.938804,0.589564
4,0.428567,0.171712,0.152156,0.083926,0.954615,0.600388
5,0.483170,0.191943,0.118303,0.067734,0.952585,0.613091
6,0.443890,0.180350,0.142016,0.076647,0.956908,0.609873
7,0.491648,0.202200,0.147134,0.081613,0.944092,0.619711
8,0.421768,0.170255,0.134765,0.072856,0.955965,0.603297
9,0.468270,0.185953,0.138217,0.075257,0.958184,0.617485


In [14]:
df_beer_results.mean(axis=0)

Lemma & Count                  0.458581
Lemma & TFIDF                  0.185948
Adjective & Count              0.143681
Adjective & TFIDF              0.078425
Dummy & BERT                   0.953094
Dummy & Senence Transformer    0.604483
dtype: float64

In [15]:
df_style_results.mean(axis=0)

Lemma & Count                  0.415609
Lemma & TFIDF                  0.160332
Adjective & Count              0.137070
Adjective & TFIDF              0.073969
Dummy & BERT                   0.952676
Dummy & Senence Transformer    0.567068
dtype: float64

In [17]:
df_beer_results.mean(axis=0) - df_style_results.mean(axis=0)

Lemma & Count                  0.042972
Lemma & TFIDF                  0.025616
Adjective & Count              0.006610
Adjective & TFIDF              0.004456
Dummy & BERT                   0.000418
Dummy & Senence Transformer    0.037415
dtype: float64